In [5]:
import ernie_airsim
ernie_airsim = ernie_airsim.ErnieAirSim()

In [6]:
command = """
假设我给你一些 XYZ 坐标作为目标。我希望你控制无人机，让它朝目标移动，同时避开前面的障碍物。无人机不应直接飞向目标，而应小步前进，同时在每一步重新评估障碍物情况。

无人机配备了距离传感器，可返回与无人机正前方最近的障碍物的距离。您可以通过调用 aw.get_distance() 来访问它。

我希望您确保只有在无人机前方距离至少 10 米的情况下才可以飞行。如果没有，请旋转无人机，直到前方至少有 10 米的间隙，d当无人机旋转后，朝着无人机当前面向的方向飞行一小段距离。当飞行一段距离后，不要忘记再次面对目标飞行。知道了吧？
请写出相关代码
"""
python_code = ernie_airsim.process(command)
print("python_code:", python_code)

python_code: import math

target_position = [x, y, z]  # 替换为实际的目标 XYZ 坐标
current_position = aw.get_drone_position()
current_yaw = aw.get_yaw()

while current_position != target_position:
    # 计算目标方向与当前方向的偏差角度
    target_yaw = math.atan2(target_position[1] - current_position[1], target_position[0] - current_position[0]) * (180 / math.pi)
    yaw_difference = target_yaw - current_yaw

    # 调整偏航角，使无人机面向目标方向
    if yaw_difference > 180:
        yaw_difference -= 360
    elif yaw_difference < -180:
        yaw_difference += 360
    aw.set_yaw(current_yaw + yaw_difference / 10)  # 分步调整偏航角，避免突然转向

    # 检查前方是否有障碍物
    distance = aw.get_distance()
    if distance < 10:
        # 旋转无人机，直到前方至少有 10 米的间隙
        while distance < 10:
            aw.set_yaw(current_yaw + 10)  # 每次旋转 10 度
            distance = aw.get_distance()

    # 朝着无人机当前面向的方向飞行一小段距离
    aw.fly_to([current_position[0] + math.cos(current_yaw * (math.pi / 180)), current_position[1] + math.sin(current_yaw * (math.pi / 180)), curr

In [7]:
command = """
写的不错，有以下小的改进：
1 调整无人机的旋转角度的时候，直接调整即可，不需要考虑突然转向造成的问题。
2 无人当前位置和目标位置不需要完成重合，只要距离小于3米就算到达了。
3 调整角度后，需要重新获得当前的偏航角current_yaw，然后再向前飞行，飞行距离可以稍微长一些，比如5米。
"""
python_code = ernie_airsim.process(command)
print("python_code:", python_code)

python_code: import math

target_position = [x, y, z]  # 替换为实际的目标 XYZ 坐标
current_position = aw.get_drone_position()
current_yaw = aw.get_yaw()

while math.dist(current_position, target_position) > 3:
    # 计算目标方向与当前方向的偏差角度
    target_yaw = math.atan2(target_position[1] - current_position[1], target_position[0] - current_position[0]) * (180 / math.pi)
    
    # 调整偏航角，使无人机面向目标方向
    aw.set_yaw(target_yaw)
    current_yaw = aw.get_yaw()  # 重新获取当前的偏航角

    # 检查前方是否有障碍物
    distance = aw.get_distance()
    if distance < 10:
        # 旋转无人机，直到前方至少有 10 米的间隙
        while distance < 10:
            aw.set_yaw(current_yaw + 10)  # 每次旋转 10 度
            distance = aw.get_distance()
            current_yaw = aw.get_yaw()  # 重新获取当前的偏航角

    # 朝着无人机当前面向的方向飞行一小段距离
    aw.fly_to([current_position[0] + math.cos(current_yaw * (math.pi / 180)) * 5, current_position[1] + math.sin(current_yaw * (math.pi / 180)) * 5, current_position[2]])
    current_position = aw.get_drone_position()

